In [1]:
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
from N2_dataprep_time import *
from random import randrange
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from math import ceil

In [4]:
class CNNLSTM(Model):
    def __init__(self):
        super().__init__()

    def setup(self, frame_in, frame_out, filters_=32, kernel_size_=5, pool_size_=4):
        self.model = Sequential()
        self.model.add(TimeDistributed(Conv1D(filters=filters_, kernel_size=kernel_size_, activation='relu'), input_shape=(None, 24,1)))
        self.model.add(TimeDistributed(MaxPooling1D(pool_size=pool_size_)))
        self.model.add(TimeDistributed(Flatten()))
        self.model.add(LSTM(40, activation='relu'))
        self.model.add(Dense(frame_out))
        opt = Adam(learning_rate=0.004)
        self.model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])

In [5]:
epochs = 2
frame_in = 24
frame_out = 2
test_ratio = 0.2
train_inp, train_oup, test_inp, test_oup = compound(setup(test_ratio))

ValueError: `TimeDistributed` Layer should be passed an `input_shape ` with at least 3 dimensions, received: (None, 24)

In [ ]:

c = CNNLSTM()
c.setup(frame_in, frame_out)
c.setData(train_inp, train_oup)
print(train_inp.shape[0], train_oup.shape[0])

In [ ]:
history = c.train(epochs_=epochs)
score = c.model.evaluate(test_inp, test_oup, verbose=0)
print(score)